In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display # Allows the use of display() for DataFrames
import scipy 
from matplotlib import pylab
import time

# Pretty display for notebooks
%matplotlib inline

## Import data to training the models

In [2]:
#Load the saved file to verify
train_data = pd.read_pickle("./data/preprocessed/preprocessed_train.pkl")
display(train_data.head())

#ave the identification number
Id = train_data['Id']

#save target varible
target = train_data['SalePrice']
display(train_data['SalePrice'].head(10))

#remove target and ID, only features
train_data.drop(labels=['SalePrice', 'Id'], axis=1, inplace=True)

display(train_data.head())

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SalePrice
0,1,4.110874,9.042040,7,5,2003,2003,6.561031,0.0,5.017280,...,0,0,1,0,0,0,0,1,0,12.247699
1,2,3.044522,9.169623,6,8,1976,1976,6.886532,0.0,5.652489,...,0,0,1,0,0,0,0,1,0,12.109016
2,3,4.110874,9.328212,7,5,2001,2002,6.188264,0.0,6.075346,...,0,0,1,0,0,0,0,1,0,12.317171
3,4,4.262680,9.164401,7,5,1915,1970,5.379897,0.0,6.293419,...,0,0,1,1,0,0,0,0,0,11.849405
4,5,4.110874,9.565284,8,5,2000,2000,6.486161,0.0,6.196444,...,0,0,1,0,0,0,0,1,0,12.429220


0    12.247699
1    12.109016
2    12.317171
3    11.849405
4    12.429220
5    11.870607
6    12.634606
7    12.206078
8    11.774528
9    11.678448
Name: SalePrice, dtype: float64

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,9.042040,7,5,2003,2003,6.561031,0.0,5.017280,6.753438,...,0,0,0,1,0,0,0,0,1,0
1,3.044522,9.169623,6,8,1976,1976,6.886532,0.0,5.652489,7.141245,...,0,0,0,1,0,0,0,0,1,0
2,4.110874,9.328212,7,5,2001,2002,6.188264,0.0,6.075346,6.825460,...,0,0,0,1,0,0,0,0,1,0
3,4.262680,9.164401,7,5,1915,1970,5.379897,0.0,6.293419,6.629363,...,0,0,0,1,1,0,0,0,0,0
4,4.110874,9.565284,8,5,2000,2000,6.486161,0.0,6.196444,7.044033,...,0,0,0,1,0,0,0,0,1,0


### Split features and target

In [3]:
#Split TRAINING SET into train and test sets
from sklearn.model_selection import train_test_split

#split dataset where 33% is testing dataset
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.33, random_state=42)

## Verify the shape
display(X_train.shape)
display(X_test.shape)

display(y_train.shape)
display(y_test.shape)

(978, 220)

(482, 220)

(978L,)

(482L,)

## Individual Models

In this section, I will try many machine learning models used to regression problems:
Linear Regression (with Ridge and Lasso regularization); Support Vector Regression; Linear
Support Vector Regression; Stochastic Gradient Descent; K Neighbors Regressor; Kernel Ridge
Regression; Decision Tree Regressor;
The method Cross Validation Score with score parameter defined as Root Mean Squared Error
will be used to evaluate and compare the result of each individual learner. Just the bests model
will be fine-tuned in the next step.


> NOTE: [Skelarn.org](http://scikit-learn.org/stable/) was used as base to write comments about methods.


## Fine-Tune Individual Models

As the promising models were defined in the last step, now I will fine-tune the most promising
models. The method to perform the fine-tune the hyper parameters will be Grid Search
CV configured according to the project requisites. Thereby, each individual model will find the
best parameters configuration.

### Croos-Valiation Score for training dataset

In [4]:
from sklearn.model_selection import cross_val_score
def rmse_CV(model, X, y):
    print "RMSE Metric"
    neg_msqe = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=0)
    rmse= np.sqrt(-neg_msqe)
    print "Scores: ",rmse 
    print 'Mean:', rmse.mean()
    print 'Standard Deviation: ', rmse.std()

### Grid Search Function: Fine-Tune

In [6]:
from sklearn.model_selection import GridSearchCV

def GridSearch(reg, parameters, X, y):
    """
    excecute grid search to determine the optimized parameters combination
    
    input
    reg: the model; parameters: dict of parameters and options; X: dataframe, training features; y:pandas series, target feature;
    """
    GSCV = GridSearchCV(reg, parameters, verbose=0, cv=5, n_jobs=-1)
    best_model = GSCV.fit(X, y)
    display(best_model)
    display(best_model.best_params_)
    
    #return the configuration of best model
    return best_model.best_estimator_

# Individual and Default Models

### Decision Tree

In [25]:
#Import model
from sklearn.tree import DecisionTreeRegressor

#Create Regressor
tree_reg = DecisionTreeRegressor()

#Fit
tree_reg.fit(X_train, y_train)

#Predict
rmse_CV(tree_reg, X_test, y_test)

RMSE Metric
Scores:  [0.22812705 0.21240967 0.19650371]
Mean: 0.21234680972875672
Standard Deviation:  0.012910250261882738


### K Neighbors Regressor

Whrite about method: Regression based on k-nearest neighbors.
The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set.

In [26]:
from sklearn.neighbors import KNeighborsRegressor

#Create the regressor
neigh = KNeighborsRegressor()

#Fit
neigh.fit(X_train, y_train)

#Predict
#display(rmse(y_test, neigh.predict(X_test)))
#Predict
rmse_CV(neigh, X_test, y_test)

RMSE Metric
Scores:  [0.2898332  0.27254986 0.24603219]
Mean: 0.2694717493515401
Standard Deviation:  0.018013663196502137


### Linear Support Vector Machine Regressor

In [27]:
from sklearn.svm import LinearSVR

#Create the regressor
lin_svr = LinearSVR()

#Fit
lin_svr.fit(X_train, y_train)

#Predict
#display(rmse(y_test, lin_svr.predict(X_test)))
rmse_CV(lin_svr, X_test, y_test)

RMSE Metric
Scores:  [0.26438393 0.39685316 0.39551933]
Mean: 0.3522521412970557
Standard Deviation:  0.06213459366926334


### Epsilon Support Vector Regression

In [28]:
from sklearn.svm import SVR

#Create Regressor
svr = SVR()

#Fit
svr.fit(X_train, y_train)

#Predict
#display(rmse(y_test, svr.predict(X_test)))
rmse_CV(svr, X_test, y_test)

RMSE Metric
Scores:  [0.31528728 0.2957921  0.26767401]
Mean: 0.2929177938961795
Standard Deviation:  0.01954400345949258


### Kernel Ridge Regressor

"Kernel ridge regression (KRR) [M2012] combines Ridge Regression (linear least squares with l2-norm regularization) with the kernel trick. It thus learns a linear function in the space induced by the respective kernel and the data. For non-linear kernels, this corresponds to a non-linear function in the original space.

The form of the model learned by KernelRidge is identical to support vector regression (SVR). However, different loss functions are used: KRR uses squared error loss while support vector regression uses \epsilon-insensitive loss, both combined with l2 regularization. In contrast to SVR, fitting KernelRidge can be done in closed-form and is typically faster for medium-sized datasets." (Kernel ridge regression, Scikit-Learn)

In [29]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics.pairwise import chi2_kernel, laplacian_kernel
#Create Regressor
ker_rid = KernelRidge()

#Fit
ker_rid.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(ker_rid, X_test, y_test)

RMSE Metric
Scores:  [0.1315246  0.1347771  0.13511582]
Mean: 0.13380583733776083
Standard Deviation:  0.0016189972512706927


### Linear Regression

Ordinary least squares Linear Regression.


In [34]:
from sklearn.linear_model import LinearRegression

#Create regressor
lin_reg = LinearRegression(n_jobs=-1)

#Fit
lin_reg.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(lin_reg, X_test, y_test)

RMSE Metric
Scores:  [0.17171291 0.14545081 0.17146583]
Mean: 0.1628765145739622
Standard Deviation:  0.012322249565294896


### Lasso 

In [35]:
from sklearn.linear_model import Lasso

#create 
lasso = Lasso()

#Fit
lasso.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(lasso, X_test, y_test)

RMSE Metric
Scores:  [0.29397053 0.28491712 0.25919599]
Mean: 0.27936121223583466
Standard Deviation:  0.01473020244739484


### Elastic Net Regressor

"ElasticNet is a linear regression model trained with L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We control the convex combination of L1 and L2 using the l1_ratio parameter."


In [36]:
from sklearn.linear_model import ElasticNet

#create regrossor
elastic = ElasticNet()

#Fit
elastic.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(elastic, X_test, y_test)

RMSE Metric
Scores:  [0.28871097 0.28027105 0.25813338]
Mean: 0.27570513216180476
Standard Deviation:  0.012894002967574909


### Multi-layer Perceptron Regressor

In [38]:
from sklearn.neural_network import MLPRegressor

#Create regressor
MLP = MLPRegressor()

#Fit
MLP.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(MLP, X_test, y_test)

RMSE Metric
Scores:  [106.09450874 121.79677806  10.76018625]
Mean: 79.55049101634147
Standard Deviation:  49.06267989415548


### Gaussian Process Regression



In [39]:
from sklearn.gaussian_process import GaussianProcessRegressor

#Create Regressor
gpr = GaussianProcessRegressor()

#Fit
gpr.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(gpr, X_test, y_test)

RMSE Metric
Scores:  [11.99715014 11.9992195  12.05912791]
Mean: 12.018499181960719
Standard Deviation:  0.02874126575553559


### Gradient Boosting Regressor

In [40]:
from sklearn.ensemble import GradientBoostingRegressor

#Create Regressor
GBR = GradientBoostingRegressor()

#Fit
GBR.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(GBR, X_test, y_test)


RMSE Metric
Scores:  [0.12638924 0.16237844 0.12653831]
Mean: 0.13843533085744136
Standard Deviation:  0.01693044332030328


### Random Forest Regressor

In [41]:
#Random Forest regressor
from sklearn.ensemble import RandomForestRegressor

#Create Regressor
rand_F = RandomForestRegressor(n_jobs=-1)

#Fit
rand_F.fit(X_train, y_train)

#Predict
#display(rmse(y_test, ker_rid.predict(X_test)))
rmse_CV(rand_F, X_test, y_test)


RMSE Metric
Scores:  [0.17697204 0.18488708 0.15988305]
Mean: 0.17391405724450704
Standard Deviation:  0.010434362521077236


## Fine-Tune Promissing Models

### Decision Tree Regressor

Write about thi method


In [7]:
#Import model
from sklearn.tree import DecisionTreeRegressor

#Create Regressor
tree_reg = DecisionTreeRegressor()

#Create Hyperparamter Search Space
criterion = ['mse', 'friedman_mse', 'mae']
splitter = ['best', 'random']
max_depth = [2, 4, 6, 8, 10, 12] #== 'None' becuase nodes are expanded until 
#all leaves are pure or until all leaves contain less than min_samples_split samples
max_features = ['auto', 'sqrt', 'log2', None]

#Create hyperparameter options
tree_hyper = dict(criterion=criterion, splitter=splitter, max_features=max_features)
#print tree_hyper


#Apply Grid Search To Determine the best model
s_time = time.time()
best_tree = GridSearch(tree_reg, tree_hyper, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))
#display(best_tree.get_params)




GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': ['auto', 'sqrt', 'log2', None], 'splitter': ['best', 'random'], 'criterion': ['mse', 'friedman_mse', 'mae']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'criterion': 'friedman_mse', 'max_features': 'auto', 'splitter': 'best'}

Training and Fine-Tune Time: -11.225000


In [8]:
print best_tree

DecisionTreeRegressor(criterion='friedman_mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')


> **NOTE:** Default Parameters

In [59]:
score_tree = rmse_CV(best_tree, X_test, y_test)

RMSE Metric
Scores:  [0.2038968  0.23366443 0.22982821]
Mean: 0.22246314630207745
Standard Deviation:  0.013221472127812437


### K Neighbors Regressor

Whrite about method: Regression based on k-nearest neighbors.
The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set.

In [60]:
from sklearn.neighbors import KNeighborsRegressor

#Create the regressor
neigh = KNeighborsRegressor()

#Create Hyperparamter Search Space
n_neighbors = [2, 4, 6, 8, 10, 12, 15, 20, 25]
weights = ['uniform', 'distance']
algorithm = ['ball_tree', 'kd_tree', 'brute']
#algorithm = ['auto']
p = [1, 2]

#Create hyperparameter options
neigh_param = dict(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm, p=p)
#print tree_hyper


#Apply Grid Search To Determine the best model
s_time = time.time()
best_neigh = GridSearch(neigh, neigh_param, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))
#display(best_tree.get_params)





GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'p': [1, 2], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute'], 'n_neighbors': [2, 4, 6, 8, 10, 12, 15, 20, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'algorithm': 'ball_tree', 'n_neighbors': 6, 'p': 1, 'weights': 'distance'}

Training and Fine-Tune Time: -32.861000


In [ ]:
print best_neigh

In [47]:
#print best_neigh.best_params_

{'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'ball_tree', 'p': 1}


In [61]:
score_neigh = rmse_CV(best_neigh, X_test, y_test)

RMSE Metric
Scores:  [0.25410762 0.24924477 0.20992898]
Mean: 0.23776045523237385
Standard Deviation:  0.01977970242938253


### Kernel Ridge Regressor

"Kernel ridge regression (KRR) [M2012] combines Ridge Regression (linear least squares with l2-norm regularization) with the kernel trick. It thus learns a linear function in the space induced by the respective kernel and the data. For non-linear kernels, this corresponds to a non-linear function in the original space.

The form of the model learned by KernelRidge is identical to support vector regression (SVR). However, different loss functions are used: KRR uses squared error loss while support vector regression uses \epsilon-insensitive loss, both combined with l2 regularization. In contrast to SVR, fitting KernelRidge can be done in closed-form and is typically faster for medium-sized datasets."

In [11]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics.pairwise import chi2_kernel, laplacian_kernel
#Create Regressor
ker_rid = KernelRidge()

#Create Hyperparamter Search Space
alpha= [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]
kernel = ['laplacian', 'rbf', 'linear', 'poly', 'sigmoid']
degree = [2,3,4,5,6,7]
gamma = np.logspace(-3, 2, 6)

#Create hyperparameter options
ker_rid_param = dict(alpha=alpha, kernel=kernel, degree=degree, gamma=gamma)

#Apply Grid Search To Determine the best model
s_time = time.time()
best_ker_rid = GridSearch(ker_rid, ker_rid_param, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))

GridSearchCV(cv=5, error_score='raise',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
      kernel_params=None),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'kernel': ['laplacian', 'rbf', 'linear', 'poly', 'sigmoid'], 'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'degree': [2, 3, 4, 5, 6, 7], 'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'alpha': 1.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'linear'}

Training and Fine-Tune Time: -924.851000


In [ ]:
print best_ker

In [12]:
score_ker_rid = rmse_CV(best_ker_rid, X_test, y_test)

RMSE Metric
Scores:  [0.1315246  0.1347771  0.13511582]
Mean: 0.13380583733776083
Standard Deviation:  0.0016189972512706927


### Stochastic Gradient Descent

whrite about model: "The class SGDRegressor implements a plain stochastic gradient descent learning routine which supports different loss functions and penalties to fit linear regression models. **SGDRegressor is well suited for regression problems with a large number of training samples (> 10.000)**, for other problems we recommend Ridge, Lasso, or ElasticNet." (Stochastic Gradient Descent, Scikit-Learn)

In [11]:
#from sklearn.linear_model import SGDRegressor

#Create a regressor
#sgd = SGDRegressor()

#Create Hyperparamter Search Space
#loss = ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
#penalty = ['l2', 'l1', 'elasticnet']
#alpha= [1.0, 0.1, 0.01, 0.001, 0.0001]
#epsilon=[0, 0, 0.01, 0.1, 0.5, 1, 2, 4]
#learning_rate = ['constant', 'optimal', 'invscaling']

#Create hyperparameter options
#sgd_param = dict(loss=loss, penalty=penalty, alpha=alpha, epsilon=epsilon, learning_rate=learning_rate)

#Apply Grid Search To Determine the best model
#s_time = time.time()
#best_sgd = GridSearch(sgd, sgd_param, X_train, y_train)
#f_time = time.time()
#print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))


#sgd_reg.fit(X, y)

C:\Users\User\Anaconda3\envs\py2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l2', 'l1', 'elasticnet'], 'epsilon': [0, 0, 0.01, 0.1, 0.5, 1, 2, 4], 'learning_rate': ['constant', 'optimal', 'invscaling'], 'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Training and Fine-Tune Time: -284.812000


In [12]:
#score_sgd = rmse_CV(best_sgd, X_train, y_train)

RMSE Metric
Scores:  [0.02460181 0.02275077 0.02262368 0.02334927 0.02355519 0.02407193
 0.02302386 0.02109088 0.02132617 0.02126972]
Mean: 0.02276632855466759
Standard Deviation:  0.001150810555570769


### Linear Regression

Ordinary least squares Linear Regression.


In [13]:
from sklearn.linear_model import LinearRegression

#Create regressor
lin_reg = LinearRegression(n_jobs=-1)

#Create Hyperparamter Search Space

#There isn't parameters to tune

best_lin_reg = lin_reg

#lin_reg.fit(X,y)

In [14]:
score_lin_reg = rmse_CV(best_lin_reg, X_test, y_test)

RMSE Metric
Scores:  [0.17171291 0.14545081 0.17146583]
Mean: 0.1628765145739622
Standard Deviation:  0.012322249565294896


### Lasso 

In [15]:
from sklearn.linear_model import Lasso

#create 
lasso = Lasso()

#Create Hyperparamter Search Space
alpha= [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]
fit_intercept = [True, False]
normalize=[False]
#precompute=[auto]
selection=['cyclic', 'random']

#Create hyperparameter options
lasso_param = dict(alpha=alpha, fit_intercept=fit_intercept, normalize=normalize, selection=selection)

#Apply Grid Search To Determine the best model
s_time = time.time()
best_lasso= GridSearch(lasso, lasso_param, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))



GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'normalize': [False], 'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0], 'selection': ['cyclic', 'random'], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'alpha': 0.0001,
 'fit_intercept': True,
 'normalize': False,
 'selection': 'random'}

Training and Fine-Tune Time: -11.879000


In [16]:
score_lasso = rmse_CV(best_lasso, X_test, y_test)

RMSE Metric
Scores:  [0.15035719 0.13107585 0.13499215]
Mean: 0.13880839390878633
Standard Deviation:  0.00832127350957245


### Elastic Net Regressor

"ElasticNet is a linear regression model trained with L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We control the convex combination of L1 and L2 using the l1_ratio parameter." (Scikit-Learn Documentation)


In [17]:
from sklearn.linear_model import ElasticNet

#create regrossor
elastic = ElasticNet()

##Create Hyperparamter Search Space
alpha= [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]
l1_ratio = [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]
fit_intercept = [True, False]
normalize=[False]
#precompute=['auto']
selection=['cyclic', 'random']

#Create hyperparameter options
elastic_param = dict(alpha=alpha, fit_intercept=fit_intercept, l1_ratio=l1_ratio, normalize=normalize, selection=selection)


#Apply Grid Search To Determine the best model
s_time = time.time()
best_elastic= GridSearch(elastic, elastic_param, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))


#elast_reg.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'normalize': [False], 'l1_ratio': [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0], 'selection': ['cyclic', 'random'], 'fit_intercept': [True, False], 'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'alpha': 0.0001,
 'fit_intercept': True,
 'l1_ratio': 1.0,
 'normalize': False,
 'selection': 'random'}

Training and Fine-Tune Time: -82.903000


In [18]:
score_lasso = rmse_CV(best_elastic, X_test, y_test)

RMSE Metric
Scores:  [0.15045421 0.13108303 0.13507172]
Mean: 0.1388696531754481
Standard Deviation:  0.008351803659718092


### Perceptron Regressor

"The Perceptron is another simple algorithm suitable for large scale learning. By default:

* It does not require a learning rate.
* It is not regularized (penalized).
* It updates its model only on mistakes.

The last characteristic implies that the Perceptron is slightly faster to train than SGD with the hinge loss and that the resulting models are sparser." ([W3cubDocs](http://docs.w3cub.com/scikit_learn/modules/linear_model/))

> **NOTE:** This optmization failed.

In [24]:
#from sklearn.linear_model import Perceptron

#create regressor
#percep = Perceptron()

#Create Hyperparamter Search Space
#penalty=['l1', 'l2', 'elasticnet']
#alpha = [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]
#Create hyperparameter options
#percep_param = dict(penalty=penalty, alpha=alpha, n_jobs=n_jobs, class_weight=class_weight)


#Apply Grid Search To Determine the best model
#s_time = time.time()
#best_percep= GridSearch(percep, percep_param, X_train, y_train)
#f_time = time.time()
#print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))




#percep.fit(X,y)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\User\Anaconda3\envs\py2\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = r'C:\Users\User\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.......................................

In [ ]:
#score_percep = rmse_CV(best_percep, X_train, y_train)

### Gaussian Process Regression

> **NOTE:** The optimization failed

In [13]:
#from sklearn.gaussian_process import GaussianProcessRegressor

#Create Regressor
#gpr = GaussianProcessRegressor()

#Create Hyperparamter Search Space
#kernel = ['rbf', 'linear', 'poly', 'sigmoid']
#alpha = [1.0, 0.1, 0.01, 0.001, 0.0001, 0.0]
#optimizer = ['fmin_l_bfgs_b', None]

#Create hyperparameter options
#gpr_param = dict(alpha=alpha, optimizer=optimizer)


#Apply Grid Search To Determine the best model
#s_time = time.time()
#best_gpr= GridSearch(gpr, gpr_param, X_train, y_train)
#f_time = time.time()
#print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))

#gpr.fit(X,y)


JoblibLinAlgError: JoblibLinAlgError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\User\Anaconda3\envs\py2\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = r'C:\Users\User\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.....................................

In [ ]:
#score_gpr = rmse_CV(best_gpr, X_test, y_test)

### Gradient Boosting Regressor

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

#Create Regressor
GBR = GradientBoostingRegressor()

#Create Hyperparamter Search Space
loss = ['ls', 'lad', 'huber', 'quantile']
criterion = ['mse']
alpha = [0.9, 0.5, 0.1, 0.05, 0.01, 0.001, 0.0001]
max_features = ['auto', 'sqrt', 'log2', None]
max_depth = [2, 4, 6, 8, 10, 12]

#Create hyperparameter options
GBR_param = dict(loss=loss,criterion=criterion, alpha=alpha, max_features=max_features, max_depth=max_depth)

#Apply Grid Search To Determine the best model
s_time = time.time()
best_GBR= GridSearch(GBR, GBR_param, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))



GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': ['auto', 'sqrt', 'log2', None], 'alpha': [0.9, 0.5, 0.1, 0.05, 0.01, 0.001, 0.0001], 'criterion': ['mse'], 'max_depth': [2, 4, 6, 8, 10, 12], 'loss': ['ls', 'lad', 'huber', 'quantile']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'alpha': 0.0001,
 'criterion': 'mse',
 'loss': 'ls',
 'max_depth': 4,
 'max_features': 'sqrt'}

Training and Fine-Tune Time: -1492.864000


In [21]:
score_GBR = rmse_CV(best_GBR, X_test, y_test)

RMSE Metric
Scores:  [0.13831081 0.15751812 0.13147632]
Mean: 0.1424350829195781
Standard Deviation:  0.011024247065513146


### Random Forest Regressor

In [22]:
#Random Forest regressor
from sklearn.ensemble import RandomForestRegressor

#Create Regressor
rand_F = RandomForestRegressor()

#Create Hyperparamter Search Space
criterion = ['mse']
max_features = ['auto', 'sqrt', 'log2', None]
max_depth = [2, 4, 6, 8, 10, 12]
n_jobs=[-1]

#Create hyperparameter options
rand_param = dict(criterion=criterion, max_features=max_features, max_depth=max_depth, n_jobs=n_jobs)

#Apply Grid Search To Determine the best model
s_time = time.time()
best_rand_F= GridSearch(rand_F, rand_param, X_train, y_train)
f_time = time.time()
print "Training and Fine-Tune Time: {:3f}".format((s_time - f_time))



GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': ['auto', 'sqrt', 'log2', None], 'n_jobs': [-1], 'criterion': ['mse'], 'max_depth': [2, 4, 6, 8, 10, 12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'criterion': 'mse', 'max_depth': 10, 'max_features': None, 'n_jobs': -1}

Training and Fine-Tune Time: -17.057000


In [23]:
score_rand = rmse_CV(best_rand_F, X_test, y_test)

RMSE Metric
Scores:  [0.17184903 0.18673306 0.15451401]
Mean: 0.17103203417143517
Standard Deviation:  0.013166050755315364
